In [86]:
import torch
import torch.nn as nn
import torch.optim as optim
import joblib
import pandas as pd
from typing import Dict, Union
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from abc import ABC, abstractmethod

class BaseModel(ABC):
    """
    Abstract base class for models.
    """
    def __init__(self) -> None:
        pass

    @abstractmethod
    def train(self, X_train: pd.DataFrame, y_train: pd.Series, epochs: int, lr: float) -> None:
        """
        Train the model.
        
        Params:
        X_train (pd.DataFrame): Training features.
        y_train (pd.Series): Training target.
        epochs (int): Number of training epochs.
        lr (float): Learning rate.
        """
        pass
    
    @abstractmethod
    def predict(self, X: pd.DataFrame) -> pd.Series:
        """
        Predict using the model.
        
        Params:
        X (pd.DataFrame): Features to predict.

        Returns:
        pd.Series: Predictions.
        """
        pass
    
    @abstractmethod
    def save(self, path: str) -> None:
        """
        Save the model to a file.
        
        Params:
        path (str): Path to save the model.
        """
        pass
    
    @abstractmethod
    def load(self, path: str) -> None:
        """
        Load the model from a file.
        
        Params:
        path (str): Path to load the model.
        """
        pass
    
    @abstractmethod
    def evaluate(self, X: pd.DataFrame, y: pd.Series) -> Dict[str, float]:
        """
        Evaluate the model.
        
        Params:
        X (pd.DataFrame): Features to evaluate.
        y (pd.Series): True values.

        Returns:
        Dict[str, float]: Evaluation metrics.
        """
        pass
    
    @abstractmethod
    def get_params(self) -> Dict[str, float]:
        """
        Get model parameters.
        
        Returns:
        Dict[str, float]: Model parameters.
        """
        pass


class MLP(nn.Module):
    """
    Simple MLP model using PyTorch.
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.l3=nn.Linear(hidden_dim,hidden_dim)
        self.l4=nn.Linear(hidden_dim,hidden_dim)
        self.l5=nn.Linear(hidden_dim,hidden_dim)
        self.l6 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = torch.relu(self.l1(x))
        x= torch.relu(self.l2(x))
        x= torch.relu(self.l3(x))
        x= torch.relu(self.l4(x))
        x= torch.relu(self.l5(x))
        
        x = self.l6(x)
        return x

class MLPModel(BaseModel):
    """
    Wrapper for the MLP model using PyTorch.
    """
    def __init__(self, input_dim: int = 23, hidden_dim: int = 128, output_dim: int = 1, id: str = None) -> None:
        """
        Initialize the MLPModel.
        
        Params:
        input_dim (int): Number of input features.
        hidden_dim (int): Number of hidden units.
        output_dim (int): Number of output units.
        id (str): Optional model ID to load a saved model.
        """
        super().__init__()
        if id is None:
            self.model = MLP(input_dim, hidden_dim, output_dim)
        else:
            self.load(id)
        
    def train(self, X_train: pd.DataFrame, y_train: pd.Series, epochs: int = 1000, lr: float = 0.001) -> None:
        """
        Train the MLP model.
        
        Params:
        X_train (pd.DataFrame): Training features.
        y_train (pd.Series): Training target.
        epochs (int): Number of training epochs.
        lr (float): Learning rate.
        """
        self.model_trained = self.model
        self.scaler=StandardScaler()
        optimizer = optim.Adam(self.model_trained.parameters(), lr=lr)
        loss_fn = nn.MSELoss()
        
        X_train = torch.tensor(X_train.values, dtype=torch.float32)
        y_train = y_train.values if isinstance(y_train, pd.Series) else y_train
        y_train=self.scaler.fit_transform(y_train.reshape(-1,1))
        y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        
        self.lr = lr
        self.epoch = epochs
        
        for epoch in range(epochs):
            self.model_trained.train()
            optimizer.zero_grad()
            y_pred = self.model_trained(X_train)
            loss = loss_fn(y_pred, y_train)
            loss.backward()
            optimizer.step()
            
            if (epoch+1) % 100 == 0:
                print(f'Epoch {epoch+1}, Loss: {loss.item()}')
        
    def predict(self, X: Union[pd.DataFrame, pd.Series]) -> pd.Series:
        """
        Predict using the MLP model.
        
        Params:
        X (Union[pd.DataFrame, pd.Series]): Features to predict.

        Returns:
        pd.Series: Predictions.
        """
        self.model_trained.eval()
        X = torch.tensor(X.values, dtype=torch.float32)
        with torch.no_grad():
            y_pred = self.model_trained(X)
        y_pred=self.scaler.inverse_transform(y_pred.numpy())
        return pd.Series(y_pred.flatten())
    
    def save(self, path: str) -> None:
        """
        Save the MLP model to a file.
        
        Params:
        path (str): Path to save the model.
        """
        torch.save(self.model_trained.state_dict(), path)
        
    def load(self, path: str) -> None:
        """
        Load the MLP model from a file.
        
        Params:
        path (str): Path to load the model.
        """
        self.model_trained = self.model
        self.model_trained.load_state_dict(torch.load(path))
        
    def evaluate(self, X: pd.DataFrame, y: pd.Series) -> Dict[str, float]:
        """
        Evaluate the MLP model.
        
        Params:
        X (pd.DataFrame): Features to evaluate.
        y (pd.Series): True values.

        Returns:
        Dict[str, float]: Evaluation metrics.
        """
        y_pred = self.predict(X)
        mae = mean_absolute_error(y, y_pred)
        mse = mean_squared_error(y, y_pred)
        self.mae_mse = {"mae": mae, "mse": mse}
        return self.mae_mse
    
    def get_params(self) -> Dict[str, float]:
        """
        Get the MLP model parameters.
        
        Returns:
        Dict[str, float]: Model parameters.
        """
        params = {"hidden_units": self.model.l1.out_features,
                  "learning_rate": self.lr, **self.mae_mse}
        self.params = params
        return params




# Assuming you have X_train, y_train, X_test, y_test as your data



In [87]:
import pandas as pd

data = pd.read_csv('data/diamonds.csv')
y=data['price']
X=data.drop('price', axis=1)
X=pd.get_dummies(X,dtype=float)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [98]:
model = MLPModel(input_dim=X_train.shape[1], hidden_dim=5, output_dim=1)
model.train(X_train, y_train, epochs=5000, lr=0.0001)
metrics = model.evaluate(X_test, y_test)
model.save('model.pth')
params = model.get_params()

Epoch 100, Loss: 1.0020232200622559
Epoch 200, Loss: 0.9988021850585938
Epoch 300, Loss: 0.9943796992301941
Epoch 400, Loss: 0.9882417917251587
Epoch 500, Loss: 0.9773734211921692
Epoch 600, Loss: 0.9583086371421814
Epoch 700, Loss: 0.9253170490264893
Epoch 800, Loss: 0.8745217323303223
Epoch 900, Loss: 0.8049335479736328
Epoch 1000, Loss: 0.723926305770874
Epoch 1100, Loss: 0.6375624537467957
Epoch 1200, Loss: 0.5530216693878174
Epoch 1300, Loss: 0.474772185087204
Epoch 1400, Loss: 0.4055730104446411
Epoch 1500, Loss: 0.34668639302253723
Epoch 1600, Loss: 0.29940178990364075
Epoch 1700, Loss: 0.26314908266067505
Epoch 1800, Loss: 0.23565377295017242
Epoch 1900, Loss: 0.21485324203968048
Epoch 2000, Loss: 0.19888974726200104
Epoch 2100, Loss: 0.18617551028728485
Epoch 2200, Loss: 0.1755368858575821
Epoch 2300, Loss: 0.16624696552753448
Epoch 2400, Loss: 0.15776914358139038
Epoch 2500, Loss: 0.1500587910413742
Epoch 2600, Loss: 0.14308661222457886
Epoch 2700, Loss: 0.136713445186615
Epo

In [97]:
params

{'hidden_units': 5,
 'learning_rate': 0.0001,
 'mae': 714.4256802978516,
 'mse': 1032427.3345071009}

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False]])